In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv
/kaggle/input/imdb-positive-train-reviews/reviews-pos.csv
/kaggle/input/imdb-negative-train-reviews/reviews-neg.csv


In [2]:
train_pos_df = pd.read_csv('/kaggle/input/imdb-positive-train-reviews/reviews-pos.csv')
train_neg_df = pd.read_csv('/kaggle/input/imdb-negative-train-reviews/reviews-neg.csv')

In [3]:
train_pos_labels = np.ones(len(train_pos_df.index))
train_neg_labels = np.zeros(len(train_neg_df.index))

In [82]:
# print(pd.concat([train_pos_df, train_neg_df]).head().to_numpy().reshape(5,)[0])

Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!


In [83]:
reviews = pd.concat([train_pos_df, train_neg_df]).to_numpy()

In [5]:
labels = np.concatenate((train_pos_labels, train_neg_labels))

In [63]:
import re

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

class ReviewProcessor:
    def remove_html_tags(self, text):
        return BeautifulSoup(text).get_text()
    
    def remove_non_alpha_or_space_characters(self, text):        
        return re.sub(r'[^a-zA-Z\s]', '', text)
        
    def remove_short_words(self, text):
        return re.sub(r'\b\w{1,2}\b', '', text)
        
    def remove_stop_words(self, text):
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words("english")) + r')\b\s*')
        return pattern.sub('', text)
    
    def lemmatize(self, text):                
        lemmatizer = WordNetLemmatizer()
        split_text = text.split()
        split_text[:] = [lemmatizer.lemmatize(word) for word in split_text]
        split_text[:] = [lemmatizer.lemmatize(word, pos='v') for word in split_text]
        return ' '.join(split_text)                
    
    def process(self, text):
        text = self.remove_html_tags(text)
        text = self.remove_non_alpha_or_space_characters(text)
        text = self.remove_short_words(text).lower()
        text = self.remove_stop_words(text)
        return self.lemmatize(text)

In [94]:
import math

from collections import Counter, defaultdict
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils.validation import check_array

class NaiveBayesClassifier(ClassifierMixin, BaseEstimator):
    def __init__(self, review_processor):
        self.review_processor = review_processor
        # TBC        
        
    # TODO: refactor
    def fit(self, X, y):                        
        # Store the classes seen during fitting                
        self.classes_, y = np.unique(y, return_inverse=True)                                
        
        # Q: Is this necessary? A: TBC
        #self.X_ = X
        #self.y_ = y        
        
        # TODO: consider extracting to another method
        # group data by class
        self.grouped_data_ = {} 
        for index, c in enumerate(self.classes_):
            self.grouped_data_[c] = X[np.asarray(y == index).nonzero()]                    
        
        
        # TODO: consider extracting to another method
        # compute log class priors
        self.log_class_priors_ = {}
        number_of_samples = X.shape[0]
        for c in self.classes_:            
            self.log_class_priors_[c] = math.log(len(self.grouped_data_[c]) / number_of_samples)        
            
        # process all reviews
        for c, data in self.grouped_data_.items():            
            for index, review in enumerate(data):                
                # Q: Is the hardcoding of zeros hacky or bad coding style in any way? A: TBC
                data[index][0] = self.review_processor.process(review[0])
        
        # construct the vocab        
        self.vocab_ = set()
        for _, data in self.grouped_data_.items():
            for review in data:
                split_review = review[0].split()
                for word in split_review:
                    self.vocab_.add(word)        
                
        
        # Construct the matrix of TF-IDF features for each class
        self.document_term_matrices_ = {}
        for c, data in self.grouped_data_.items():
            reshaped_data = data.reshape(len(data))
            self.document_term_matrices_[c] = TfidfVectorizer(vocabulary=vocabs[c]).fit_transform(reshaped_data).toarray()                            
                            
        self.class_word_counts_ = {}
        for c, data in self.grouped_data_.items():
            self.class_word_counts_[c] = defaultdict(lambda: 0)
            for review in data:
                split_review = review[0].split()
                # TODO: consider creating a counts dictionary using Counter,
                # instead of incrementing by one each time
                for word in split_review:
                    self.class_word_counts_[c][word] += 1
        
        
        #TBC
        
        return self
        
    def predict(self, X):
        # TODO: Consider checking if has been fitted. See:
        # https://scikit-learn.org/stable/developers/develop.html#rolling-your-own-estimator
        # and
        # https://github.com/scikit-learn-contrib/project-template/blob/master/skltemplate/_template.py
        
        # Q: is this necessary? See:
        # https://scikit-learn.org/stable/developers/develop.html#input-validation
        X = check_array(X)
        
        vocab_size = len(self.vocab_)
        
        for review in X:
            # Q: Is it possible to access self.classes_ here?
            for c in self.classes_:                
                class_posterior = self.log_class_priors_[c]
                processed_review = self.review_processor.process(review[0])
                split_processed_review = processed_review.split() # TODO: consider splitting in previous line
                word_counts = Counter(split_processed_review)
                number_of_class_reviews = len(self.grouped_data_[c])
                # TODO: consider extracting to a method to reduce nesting and improve readability
                for index, word in enumerate(self.vocab_):
                    word_count = word_counts[word]
                    laplace_probability = (self.class_word_counts_[c][word] + 1) / (number_of_class_reviews + vocab_size)
                    class_posterior += (word_count * math.log(laplace_probability))
        
        
        #TBC
        
        #return self.classes_[TBC]

In [95]:
clf = NaiveBayesClassifier(ReviewProcessor())
clf, document_term_matrix = clf.fit(reviews, labels)

(12500, 76014) 

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [25]:
# Check if estimator passes sklearn tests
from sklearn.utils.estimator_checks import check_estimator
check_estimator(NaiveBayesClassifier()) # pass generate_only=True to run all checks instead of failing at the first error

ValueError: Expected array-like (array or non-string sequence), got None

In [ ]:
def scorer(estimator, X, y):
    #TBC
    # return single value TBC

In [ ]:
naive_bayes_classifier = NaiveBayesClassifier()

In [ ]:
from sklearn.model_selection import RepeatedKFold, cross_validate

# Might be simpler to use cross_val_score
cv_model = cross_validate(
    naive_bayes_classifier, reviews, labels, cv=RepeatedKFold(n_splits=5, n_repeats=3),
    return_estimator=True, n_jobs=-1
)

In [ ]:
# load data

# reviews_df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
# reviews_df.head()

In [ ]:
# from bs4 import BeautifulSoup

# # Remove HTML tags from all reviews
# reviews_df['review'] = [BeautifulSoup(review).get_text() for review in reviews_df['review']]
# reviews_df.head(20)


In [ ]:
# # remove punctuation and numbers but not spaces
# import re

# pattern = re.compile(r'[^a-zA-Z\s]') 
# reviews_df['review'] = [pattern.sub('', review) for review in reviews_df['review']]

In [ ]:
# reviews_df.head(20)

In [ ]:
# # remove words of length less than 3
# reviews_df['review'] = [re.sub(r'\b\w{1,2}\b', '', review) for review in reviews_df['review']]

In [ ]:
# reviews_df.head(20)

In [ ]:
# remove stop words
# from nltk.corpus import stopwords

# pattern = re.compile(r'\b(' + r'|'.join(stopwords.words("english")) + r')\b\s*')
# reviews_df['review'] = [pattern.sub('', review) for review in reviews_df['review']]
# reviews_df['review'] = [review.split() for review in reviews_df['review']]

# for review in reviews_df['review']:
#     review_as_list = review.split()
#     review_as_list[:] = [pattern.sub('', word) for word ]

In [ ]:
# reviews_df.head(20)

In [ ]:
# print(stopwords.words("english"))

In [ ]:
# lowercase everything
# reviews_df['review'] = [review.lower() for review in reviews_df['review']]

In [ ]:
# reviews_df.head(20)

In [ ]:
# lemmatize
# from nltk.stem.wordnet import WordNetLemmatizer

# # TODO: figure out if there's a cleaner and/or more efficient way of doing this
# lemmatizer = WordNetLemmatizer()
# for index, review in enumerate(reviews_df['review']):
#     split_review = review.split()
#     split_review[:] = [lemmatizer.lemmatize(word) for word in split_review]
#     split_review[:] = [lemmatizer.lemmatize(word, pos='v') for word in split_review]
#     reviews_df['review'][index] = ' '.join(split_review)

In [ ]:
# reviews_df.head(20)